In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from time import mktime as mktime
from datetime import datetime
from shapely.geometry import Point, Polygon
from math import cos, asin, sqrt, pi
import math

TIME_FORMAT = "%Y-%m-%d %H:%M:%S"
# calculate the distance between two points specified by latitude and longitude
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
def distance(row):
    lat1 = row['LAT']
    lon1 = row['LON']
    lat2 = row['LAT-shift']
    lon2 = row['LON-shift']
    r = 6371 # km
    p = pi / 180

    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 2 * r * asin(sqrt(a))


def isValidDate(date):
    try:
        if ":" in date:
          return True
    except:
        return False
def distance_accumulate():
  # 3.2日任务
  print("这段代码计算距离的和")

def seconds_since_midnight(row):
  dateTime = pd.to_datetime(row['BaseDateTime'],
               TIME_FORMAT)
  # print(dateTime)
  # print(type(dateTime))
  # time = pd.Series.dt.to_pydatetime(dateTime)
  time = dateTime
  new_time_1 = pd.to_datetime(time)
  new_time_2 = pd.to_datetime(time.replace(hour=0, minute=0, second=0, microsecond=0))

  s = (new_time_1 - new_time_2).total_seconds()
  return math.ceil(s)
  # print(s)
  # print(type(s))
  # d = datetime.strptime(str(row['BaseDateTime']), TIME_FORMAT)
  # return time.mktime(d.timetuple())
  # return new_time.dt.total_seconds()
  # return time - time.replace(hour=0, minute=0, second=0, microsecond=0).total_seconds()

# 统一速度单位到m/s
def get_speed(row):
  U = 0.5144
  ret = row['SOG'] * U
  return ret

def speed_Enlarge(row):
  ret = int(row['speed'] * 1000)
  return ret
def Calculated_speed_enlarge(row):
  ret = int(row['speed_calculated'] * 1000)
  return ret

def get_calculated_speed(row):
  if row['x_i'] is not None and row['x_i']:
    ret = row['distance-diff'] * 1000 / row['x_i']
  else:
    ret = None
  return ret

# 数据分组
def data_analysis():
  df = pd.read_csv('sample_data/smallaisdata.csv')
  # df['correct'] = pd.to_datetime(df['BaseDateTime'],errors='coerce')
  # 过滤器 选取相关列
  new_df = df.filter(items=['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG','COG','Heading'])
  new_df['ErrorPoint'] = False
  # 去除异常值
  cleaned_df = new_df.dropna()
  for x in cleaned_df.index:
    if cleaned_df.loc[x, "MMSI"] < 100000000 or cleaned_df.loc[x, "MMSI"] >= 1000000000:
      cleaned_df.drop(x, inplace = True)

  # print(cleaned_df)



# 下午任务： 24.01.20 切割轨道(1.21问stackoverflow)(建立新表格 依次合并)，按照5个为基准过滤(finished)，计算并移除异常轨迹点 (建立时间戳索引, 进行地图标注)
# 24.01.21 移除异常轨迹点
  df = cleaned_df.sort_values(by=['MMSI', 'BaseDateTime'])
  df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])  # Convert 'BaseDateTime' to datetime format
  df['new-time'] = df.groupby('MMSI')['BaseDateTime'].shift(-1)
  # Calculate the time difference
  s = df['BaseDateTime']

  df['Time-diff'] = (s.groupby(df['MMSI']).diff(-1)
                    .mul(-1).dt.total_seconds().div(3600)
                  )
  # Assign a group label based on the time difference



  # 使用中间需要加一段南师大硕士论文的方法检测并去除异常点异常点
  df1 = df[['LAT', 'LON']]
  # 本段加一段去除异常点的方法



  # df['GROUP'] = df.groupby('MMSI')['Time-diff'].transform(assign_group).cumsum()
  df['Group'] = df['Time-diff'].fillna(np.inf).ge(2).cumsum()
  # df = df.filter(df.groupby(['MMSI','Group']).size() < 4)

  # Drop the 'new-time' column if you don't need it anymore
  df = df.drop(columns=['new-time'])


  df = df.groupby('Group').filter(lambda x: len(x) > 4)


  # 过滤掉size小于5的点

  # v = s/t add a column    speed 这个好弄 add a column ErrorPoint(0 means ERROR 1 means right)
  # 选取矩阵[(group,TimeStamp), LAT,LON,Speed, Calculated_speed]

  # [MMSI,Group, TimeStamp, LAT, LON, Speed, Calculated_speed(s/t), ErrorPoint]

  # Time-diff -> hours , Distance -> distance function
  # 1 knots = 1.85200 kilometers per hour
  df['ErrorPoint'] = 0



  # x_i = seconds_since_midnight = (now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()


  # 2024.1.30日任务 将datetime数据和累加部分代码写完
  df['x_i'] = df.apply(seconds_since_midnight, axis=1) # function debug
  # print(df)
  df = df.sort_values(by=['Group', 'BaseDateTime'])
  df = df.dropna(how='any',axis=0)
  # df.to_csv('out1.csv')



  df['LAT-shift'] = (df['LAT'].groupby(df['Group']).shift(1)
                  )
  # df.groupby(['helpkey','sym']):
  df['LON-shift'] = (df['LON'].groupby(df['Group']).shift(1)
                      )
  df['distance-diff'] = df.apply(distance, axis=1) # unit: km
  # display(df)

  df['speed'] = df.apply(get_speed,axis=1) # to m/s
  # speed -> speed * 0.5144
  df['speed_calculated'] = df.apply(get_calculated_speed,axis=1)
  # df1 = df
  # df1.to_csv('out1.csv')

  df = df.drop('Time-diff', axis=1)
  df = df.drop('LAT-shift', axis=1)
  df = df.drop('LON-shift', axis=1)

  # de下bug

  # display(df)

  # 1 将df转成numpy矩阵，进行遍历并标注错误点

  # 2 计算距离之和，进行可视化matpoltlib


  # 3 根据置信度计算停留点

  #
  # df1 = df[['a', 'b']]
  # 按顺序给数据编号
  df['ID'] = df.index
  df1 = df.dropna(how='any',axis=0)
  # display(df1)

  # print(df.dtypes)



  # speed放大3倍 向下取整
  df1['speed_enlarged'] = df1.apply(speed_Enlarge,axis=1)
  # 查询同一组中的最大速度，如果计算出的速度大于平均速度，则标记为错误点。3.2任务
  # 对于标记出的错误点进行清洗，截图写入论文中

  df1['calculated_speed_enlarged'] = df1.apply(Calculated_speed_enlarge,axis=1)

  m = df1[['ID','Group','speed_enlarged', 'calculated_speed_enlarged','ErrorPoint']].to_numpy()

  # ['ID', 'Group', 'speed_enlarged','calculated_speed_enlarged','ErrorPoint']
  # 目的是更新ErrorPoint，错误点标1


  df['seconds_since_midnight'] = df.apply(seconds_since_midnight,axis = 1) # 没问题
  df['distance-diff'].fillna(0, inplace=True) # 将NaN改成0
  # 横坐标和纵坐标都有了
  df['distance_sum'] =





  # display(df2)
  # df['distance_sum'] =










  # 将dataframe转成numpy矩阵进行处理

  # df['speed_calculated'] =  # s/t

  # distance-diff :km

  # m = np.array([0,1,2])
  # for row in m:
    # print(row)


  display(df)

  # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  # print(df) # display all the dataframe





  # 3.2日任务，完成曲线图，尝试标记停留点





In [ ]:
data_analysis()

<ipython-input-5-91a804d721f9>:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['speed_enlarged'] = df1.apply(speed_Enlarge,axis=1)
<ipython-input-5-91a804d721f9>:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['calculated_speed_enlarged'] = df1.apply(Calculated_speed_enlarge,axis=1)


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,ErrorPoint,Group,x_i,distance-diff,speed,speed_calculated,ID,seconds_since_midnight
3024,369493300.0,2022-12-08 00:00:08,46.25190,-124.09223,4.5,63.7,511.0,0,1,8,0.000000,2.31480,NaN,3024,8
3025,369493300.0,2022-12-08 00:01:10,46.25232,-124.09088,4.5,42.5,511.0,0,1,70,0.113823,2.31480,1.626048,3025,70
3026,369493300.0,2022-12-08 00:02:17,46.25263,-124.08937,3.4,114.7,511.0,0,1,137,0.121112,1.74896,0.884028,3026,137
3027,369493300.0,2022-12-08 00:03:18,46.25296,-124.08805,2.8,78.4,511.0,0,1,198,0.107923,1.44032,0.545065,3027,198
3029,369493300.0,2022-12-08 00:04:20,46.25319,-124.08677,3.9,62.1,511.0,0,1,260,0.101686,2.00616,0.391099,3029,260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,985346207.0,2022-12-08 00:01:17,29.68807,-91.21740,0.2,314.5,126.0,0,976,77,0.000000,0.10288,0.000000,5,77
7,985346207.0,2022-12-08 00:02:26,29.68807,-91.21740,0.0,319.0,125.0,0,976,146,0.000000,0.00000,0.000000,7,146
6,985346207.0,2022-12-08 00:03:27,29.68808,-91.21740,0.0,310.5,126.0,0,976,207,0.001111,0.00000,0.005368,6,207
8,985346207.0,2022-12-08 00:04:37,29.68808,-91.21740,0.0,295.8,126.0,0,976,277,0.000000,0.00000,0.000000,8,277


In [ ]:
# 本段代码在地图上标注数据点
import plotly.express as px
import pandas as pd

def AIS_point_plot():
  color_scale = [(0, 'orange'), (1,'red')]
  df = pd.read_csv('sample_data/smallaisdata.csv')
  fig = px.scatter_mapbox(df,
                        lat="LAT",
                        lon="LON",
                        hover_name="MMSI",
                        hover_data=["MMSI"],
                        color_continuous_scale=color_scale,
                        zoom=8,
                        height=800,
                        width=800)
  # 对数据点进行核密度分析


  fig.update_layout(mapbox_style="open-street-map")
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.show()



AIS_point_plot()